# Load Pretrained Model 

In [1]:
import torch
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from nni.compression.pytorch.speedup import ModelSpeedup
from nni.compression.pytorch.utils import count_flops_params
import time

from mnist_model import Net, train, test, device, optimizer_scheduler_generator, trainer

# Load pretrained model
model = torch.load("mnist_cnn.pt")
model.eval()

# show the model stbructure, note that pruner will wrap the model layer.
print(model)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout(p=0.25, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)


### Performance and statistics of pre-trained model 

In [2]:
start = time.time()

pre_best_acc = test(model, device)
pre_test_time = time.time() - start

pre_flops, pre_params, _ = count_flops_params(model, torch.randn([3, 1, 28, 28]).to(device))
print(f'Pretrained model FLOPs {pre_flops/1e6:.2f} M, #Params: {pre_params/1e6:.2f}M, Accuracy: {pre_best_acc: .2f}%, Test-time: {pre_test_time: .4f}s')


Test set: Average loss: 0.0267, Accuracy: 9919/10000 (99.19%)

+-------+-------+--------+----------------+-----------------+-----------------+----------+---------+
| Index | Name  |  Type  |  Weight Shape  |    Input Size   |   Output Size   |  FLOPs   | #Params |
+-------+-------+--------+----------------+-----------------+-----------------+----------+---------+
|   0   | conv1 | Conv2d | (32, 1, 3, 3)  |  (3, 1, 28, 28) | (3, 32, 26, 26) |  194688  |   320   |
|   1   | conv2 | Conv2d | (64, 32, 3, 3) | (3, 32, 26, 26) | (3, 64, 24, 24) | 10616832 |  18496  |
|   2   | fc1   | Linear |  (128, 9216)   |    (3, 9216)    |     (3, 128)    | 1179648  | 1179776 |
|   3   | fc2   | Linear |   (10, 128)    |     (3, 128)    |     (3, 10)     |   1280   |   1290  |
+-------+-------+--------+----------------+-----------------+-----------------+----------+---------+
FLOPs total: 11992448
#Params total: 1199882
Pretrained model FLOPs 11.99 M, #Params: 1.20M, Accuracy:  99.19%, Test-time:  1.72

# Pruning Model  ADMMPruner

In [12]:
from nni.compression.pytorch.pruning import ADMMPruner
from nni.compression.pytorch.pruning import ActivationMeanRankPruner
from nni.compression.pytorch.speedup import ModelSpeedup
import nni

import torch.nn.functional as F

def pruner_function(config_list):

    model = torch.load("mnist_cnn.pt")
    model.eval()

    traced_optimizer = nni.trace(optim.Adadelta)(model.parameters(), lr=1.0)
    criterion = F.nll_loss
    
    # Using ADMMPruner to prune the model and generate the masks.
    pruner = ADMMPruner(model, config_list, trainer, traced_optimizer, criterion, iterations=5, training_epochs=1, granularity='coarse-grained')
    
    # pruner = ActivationMeanRankPruner(model, config_list, trainer, traced_optimizer, criterion, training_batches=20)
    
    # show the wrapped model structure, `PrunerModuleWrapper` have wrapped the layers that configured in the config_list.
    #print(model)

    # compress the model and generate the masks
    _, masks = pruner.compress()

    # show the masks sparsity
    print("Showing the masks sparsity")
    for name, mask in masks.items():
        print(name, ' sparsity : ', '{:.2}'.format(mask['weight'].sum() / mask['weight'].numel()))


    # need to unwrap the model, if the model is wrapped before speedup
    pruner._unwrap_model()

    # speedup the model, for more information about speedup, please refer :doc:`pruning_speedup`.
    ModelSpeedup(model, torch.rand(3, 1, 28, 28).to(device), masks).speedup_model()

    #print("Model after speedup")
    #print(model)

    optimizer, scheduler = optimizer_scheduler_generator(model)
    
    # fine- tuning model compacted model
    # tuning and evaluate the model on MNIST dataset
    total_epoch = 3
    
    for epoch in range(1, total_epoch + 1):
        train(model, device, optimizer=optimizer, epoch=epoch)
        test(model, device)
        scheduler.step()
        
    return model

In [11]:
def Perfomance_function(model):
    print("Model after speedup")
    print(model)
    
    start = time.time()
    best_acc = test(model, device)
    test_time = time.time() - start

    flops, params, _ = count_flops_params(model, torch.randn([3, 1, 28, 28]).to(device))

    print(f'Pretrained model FLOPs {pre_flops/1e6:.2f} M, #Params: {pre_params/1e6:.2f}M, Accuracy: {pre_best_acc: .2f}%, , Test-time: {pre_test_time: .4f}s')
    print(f'Finetuned model FLOPs {flops/1e6:.2f} M, #Params: {params/1e6:.2f}M, Accuracy: {best_acc: .2f}%, Test-time: {test_time: .4f}s, Speed-up: {pre_test_time/test_time: .2f}x')

## ADMM Configuration 1

In [13]:
config_list = [{
    'sparsity_per_layer': 0.50,
    'op_types': ['Linear', 'Conv2d']
}, {
    'exclude': True,
    'op_names': ['fc2']
}]


pruned_model = pruner_function(config_list=config_list)

[2022-10-03 19:54:19] WARNING: The old API trainer,traced_optimizer,criterion,iterations,training_epochs,granularity will be deprecated after NNI v3.0, please using the new one evaluator,iterations,training_epochs,granularity
[2022-10-03 19:54:19] ======= ADMM Iteration 0 Start =======
Train Epoch: 1 [0/60000 (0%)]	Loss: 0.006760
Train Epoch: 1 [640/60000 (1%)]	Loss: 0.028399
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.042663
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.018933
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.196756
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.037106
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.093251
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.001200
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.002165
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.005397
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.002615
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.013063
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.006169
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.182564
Train Epoch: 1 [8960/60000 (15%)]	Loss: 

Train Epoch: 1 [40960/60000 (68%)]	Loss: 0.072487
Train Epoch: 1 [41600/60000 (69%)]	Loss: 0.004401
Train Epoch: 1 [42240/60000 (70%)]	Loss: 0.087941
Train Epoch: 1 [42880/60000 (71%)]	Loss: 0.016221
Train Epoch: 1 [43520/60000 (72%)]	Loss: 0.053397
Train Epoch: 1 [44160/60000 (74%)]	Loss: 0.005047
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.010225
Train Epoch: 1 [45440/60000 (76%)]	Loss: 0.006717
Train Epoch: 1 [46080/60000 (77%)]	Loss: 0.042792
Train Epoch: 1 [46720/60000 (78%)]	Loss: 0.006889
Train Epoch: 1 [47360/60000 (79%)]	Loss: 0.030331
Train Epoch: 1 [48000/60000 (80%)]	Loss: 0.001094
Train Epoch: 1 [48640/60000 (81%)]	Loss: 0.010923
Train Epoch: 1 [49280/60000 (82%)]	Loss: 0.029677
Train Epoch: 1 [49920/60000 (83%)]	Loss: 0.020485
Train Epoch: 1 [50560/60000 (84%)]	Loss: 0.088996
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.082213
Train Epoch: 1 [51840/60000 (86%)]	Loss: 0.010566
Train Epoch: 1 [52480/60000 (87%)]	Loss: 0.119913
Train Epoch: 1 [53120/60000 (88%)]	Loss: 0.082115


Train Epoch: 1 [24960/60000 (42%)]	Loss: 0.005451
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.050184
Train Epoch: 1 [26240/60000 (44%)]	Loss: 0.003139
Train Epoch: 1 [26880/60000 (45%)]	Loss: 0.011647
Train Epoch: 1 [27520/60000 (46%)]	Loss: 0.011507
Train Epoch: 1 [28160/60000 (47%)]	Loss: 0.002835
Train Epoch: 1 [28800/60000 (48%)]	Loss: 0.001309
Train Epoch: 1 [29440/60000 (49%)]	Loss: 0.011711
Train Epoch: 1 [30080/60000 (50%)]	Loss: 0.002857
Train Epoch: 1 [30720/60000 (51%)]	Loss: 0.084022
Train Epoch: 1 [31360/60000 (52%)]	Loss: 0.003576
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.024092
Train Epoch: 1 [32640/60000 (54%)]	Loss: 0.033829
Train Epoch: 1 [33280/60000 (55%)]	Loss: 0.054936
Train Epoch: 1 [33920/60000 (57%)]	Loss: 0.031430
Train Epoch: 1 [34560/60000 (58%)]	Loss: 0.173166
Train Epoch: 1 [35200/60000 (59%)]	Loss: 0.080277
Train Epoch: 1 [35840/60000 (60%)]	Loss: 0.015929
Train Epoch: 1 [36480/60000 (61%)]	Loss: 0.001188
Train Epoch: 1 [37120/60000 (62%)]	Loss: 0.118104


[2022-10-03 19:55:42] Update mask for dropout2
[2022-10-03 19:55:42] Update mask for fc2
[2022-10-03 19:55:42] Update mask for .aten::log_softmax.11
[2022-10-03 19:55:42] Update the indirect sparsity for the .aten::log_softmax.11
[2022-10-03 19:55:42] Update the indirect sparsity for the fc2
[2022-10-03 19:55:42] Update the indirect sparsity for the dropout2
[2022-10-03 19:55:42] Update the indirect sparsity for the .aten::relu.10
[2022-10-03 19:55:42] Update the indirect sparsity for the fc1
[2022-10-03 19:55:42] Update the indirect sparsity for the .aten::flatten.9
[2022-10-03 19:55:42] Update the indirect sparsity for the dropout1
[2022-10-03 19:55:42] Update the indirect sparsity for the .aten::max_pool2d.8
[2022-10-03 19:55:42] Update the indirect sparsity for the .aten::relu.7
[2022-10-03 19:55:42] Update the indirect sparsity for the conv2
[2022-10-03 19:55:42] Update the indirect sparsity for the .aten::relu.6
[2022-10-03 19:55:42] Update the indirect sparsity for the conv1
[20

Train Epoch: 2 [20480/60000 (34%)]	Loss: 0.008165
Train Epoch: 2 [21120/60000 (35%)]	Loss: 0.039322
Train Epoch: 2 [21760/60000 (36%)]	Loss: 0.078207
Train Epoch: 2 [22400/60000 (37%)]	Loss: 0.060165
Train Epoch: 2 [23040/60000 (38%)]	Loss: 0.039575
Train Epoch: 2 [23680/60000 (39%)]	Loss: 0.063455
Train Epoch: 2 [24320/60000 (41%)]	Loss: 0.025801
Train Epoch: 2 [24960/60000 (42%)]	Loss: 0.017716
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.079669
Train Epoch: 2 [26240/60000 (44%)]	Loss: 0.057008
Train Epoch: 2 [26880/60000 (45%)]	Loss: 0.091605
Train Epoch: 2 [27520/60000 (46%)]	Loss: 0.044033
Train Epoch: 2 [28160/60000 (47%)]	Loss: 0.127389
Train Epoch: 2 [28800/60000 (48%)]	Loss: 0.039346
Train Epoch: 2 [29440/60000 (49%)]	Loss: 0.049523
Train Epoch: 2 [30080/60000 (50%)]	Loss: 0.016962
Train Epoch: 2 [30720/60000 (51%)]	Loss: 0.063095
Train Epoch: 2 [31360/60000 (52%)]	Loss: 0.023560
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.032076
Train Epoch: 2 [32640/60000 (54%)]	Loss: 0.083329



Test set: Average loss: 0.0371, Accuracy: 9896/10000 (98.96%)

+-------+-------+--------+----------------+-----------------+-----------------+---------+---------+
| Index | Name  |  Type  |  Weight Shape  |    Input Size   |   Output Size   |  FLOPs  | #Params |
+-------+-------+--------+----------------+-----------------+-----------------+---------+---------+
|   0   | conv1 | Conv2d | (16, 1, 3, 3)  |  (3, 1, 28, 28) | (3, 16, 26, 26) |  97344  |   160   |
|   1   | conv2 | Conv2d | (32, 16, 3, 3) | (3, 16, 26, 26) | (3, 32, 24, 24) | 2654208 |   4640  |
|   2   | fc1   | Linear |   (64, 4608)   |    (3, 4608)    |     (3, 64)     |  294912 |  294976 |
|   3   | fc2   | Linear |    (10, 64)    |     (3, 64)     |     (3, 10)     |   640   |   650   |
+-------+-------+--------+----------------+-----------------+-----------------+---------+---------+
FLOPs total: 3047104
#Params total: 300426
Pretrained model FLOPs 11.99 M, #Params: 1.20M, Accuracy:  99.19%, , Test-time:  1.7263s
Fine

In [14]:
Perfomance_function(pruned_model)

Model after speedup
Net(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout(p=0.25, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=4608, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=10, bias=True)
)

Test set: Average loss: 0.0371, Accuracy: 9896/10000 (98.96%)

+-------+-------+--------+----------------+-----------------+-----------------+---------+---------+
| Index | Name  |  Type  |  Weight Shape  |    Input Size   |   Output Size   |  FLOPs  | #Params |
+-------+-------+--------+----------------+-----------------+-----------------+---------+---------+
|   0   | conv1 | Conv2d | (16, 1, 3, 3)  |  (3, 1, 28, 28) | (3, 16, 26, 26) |  97344  |   160   |
|   1   | conv2 | Conv2d | (32, 16, 3, 3) | (3, 16, 26, 26) | (3, 32, 24, 24) | 2654208 |   4640  |
|   2   | fc1   | Linear |   (64, 4608)   |    (3, 4608)    |     (3, 64)    

## ADMM Configuration 2

In [16]:
config_list = [{
    'op_types': ['Conv2d'],
    'total_sparsity': 0.5
    }, {
    'op_names': ['Linear'],
    'total_sparsity': 0.8
    },
    {
    'exclude': True,
    'op_names': ['fc2']
}]


pruned_model = pruner_function(config_list=config_list)

[2022-10-03 20:06:37] WARNING: The old API trainer,traced_optimizer,criterion,iterations,training_epochs,granularity will be deprecated after NNI v3.0, please using the new one evaluator,iterations,training_epochs,granularity
[2022-10-03 20:06:37] WARNING: op_names ['Linear'] not found in model
[2022-10-03 20:06:37] WARNING: op_names ['Linear'] not found in model
[2022-10-03 20:06:37] ======= ADMM Iteration 0 Start =======
Train Epoch: 1 [0/60000 (0%)]	Loss: 0.003347
Train Epoch: 1 [640/60000 (1%)]	Loss: 0.064986
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.118320
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.043590
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.021751
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.046671
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.065379
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.010360
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.042419
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.104355
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.002413
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.13337

Train Epoch: 1 [39040/60000 (65%)]	Loss: 0.002895
Train Epoch: 1 [39680/60000 (66%)]	Loss: 0.007834
Train Epoch: 1 [40320/60000 (67%)]	Loss: 0.041079
Train Epoch: 1 [40960/60000 (68%)]	Loss: 0.155944
Train Epoch: 1 [41600/60000 (69%)]	Loss: 0.066339
Train Epoch: 1 [42240/60000 (70%)]	Loss: 0.075255
Train Epoch: 1 [42880/60000 (71%)]	Loss: 0.037815
Train Epoch: 1 [43520/60000 (72%)]	Loss: 0.032026
Train Epoch: 1 [44160/60000 (74%)]	Loss: 0.176497
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.075605
Train Epoch: 1 [45440/60000 (76%)]	Loss: 0.030503
Train Epoch: 1 [46080/60000 (77%)]	Loss: 0.195375
Train Epoch: 1 [46720/60000 (78%)]	Loss: 0.003453
Train Epoch: 1 [47360/60000 (79%)]	Loss: 0.074928
Train Epoch: 1 [48000/60000 (80%)]	Loss: 0.036460
Train Epoch: 1 [48640/60000 (81%)]	Loss: 0.001974
Train Epoch: 1 [49280/60000 (82%)]	Loss: 0.000552
Train Epoch: 1 [49920/60000 (83%)]	Loss: 0.097123
Train Epoch: 1 [50560/60000 (84%)]	Loss: 0.006524
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.009016


Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.005199
Train Epoch: 1 [23680/60000 (39%)]	Loss: 0.011936
Train Epoch: 1 [24320/60000 (41%)]	Loss: 0.018511
Train Epoch: 1 [24960/60000 (42%)]	Loss: 0.017166
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.046548
Train Epoch: 1 [26240/60000 (44%)]	Loss: 0.170707
Train Epoch: 1 [26880/60000 (45%)]	Loss: 0.166011
Train Epoch: 1 [27520/60000 (46%)]	Loss: 0.115726
Train Epoch: 1 [28160/60000 (47%)]	Loss: 0.001464
Train Epoch: 1 [28800/60000 (48%)]	Loss: 0.119669
Train Epoch: 1 [29440/60000 (49%)]	Loss: 0.000598
Train Epoch: 1 [30080/60000 (50%)]	Loss: 0.002161
Train Epoch: 1 [30720/60000 (51%)]	Loss: 0.001876
Train Epoch: 1 [31360/60000 (52%)]	Loss: 0.028390
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.011230
Train Epoch: 1 [32640/60000 (54%)]	Loss: 0.010820
Train Epoch: 1 [33280/60000 (55%)]	Loss: 0.009139
Train Epoch: 1 [33920/60000 (57%)]	Loss: 0.132059
Train Epoch: 1 [34560/60000 (58%)]	Loss: 0.002860
Train Epoch: 1 [35200/60000 (59%)]	Loss: 0.010966


[2022-10-03 20:07:56] Update mask for fc1
[2022-10-03 20:07:56] Update mask for .aten::relu.10
[2022-10-03 20:07:56] Update mask for dropout2
[2022-10-03 20:07:56] Update mask for fc2
[2022-10-03 20:07:56] Update mask for .aten::log_softmax.11
[2022-10-03 20:07:56] Update the indirect sparsity for the .aten::log_softmax.11
[2022-10-03 20:07:56] Update the indirect sparsity for the fc2
[2022-10-03 20:07:56] Update the indirect sparsity for the dropout2
[2022-10-03 20:07:56] Update the indirect sparsity for the .aten::relu.10
[2022-10-03 20:07:56] Update the indirect sparsity for the fc1
[2022-10-03 20:07:56] Update the indirect sparsity for the .aten::flatten.9
[2022-10-03 20:07:56] Update the indirect sparsity for the dropout1
[2022-10-03 20:07:56] Update the indirect sparsity for the .aten::max_pool2d.8
[2022-10-03 20:07:56] Update the indirect sparsity for the .aten::relu.7
[2022-10-03 20:07:56] Update the indirect sparsity for the conv2
[2022-10-03 20:07:56] Update the indirect spar

Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.035134
Train Epoch: 2 [19840/60000 (33%)]	Loss: 0.042199
Train Epoch: 2 [20480/60000 (34%)]	Loss: 0.086646
Train Epoch: 2 [21120/60000 (35%)]	Loss: 0.034770
Train Epoch: 2 [21760/60000 (36%)]	Loss: 0.017385
Train Epoch: 2 [22400/60000 (37%)]	Loss: 0.015602
Train Epoch: 2 [23040/60000 (38%)]	Loss: 0.015809
Train Epoch: 2 [23680/60000 (39%)]	Loss: 0.014872
Train Epoch: 2 [24320/60000 (41%)]	Loss: 0.010124
Train Epoch: 2 [24960/60000 (42%)]	Loss: 0.007516
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.016188
Train Epoch: 2 [26240/60000 (44%)]	Loss: 0.153169
Train Epoch: 2 [26880/60000 (45%)]	Loss: 0.091117
Train Epoch: 2 [27520/60000 (46%)]	Loss: 0.007027
Train Epoch: 2 [28160/60000 (47%)]	Loss: 0.105933
Train Epoch: 2 [28800/60000 (48%)]	Loss: 0.003257
Train Epoch: 2 [29440/60000 (49%)]	Loss: 0.100779
Train Epoch: 2 [30080/60000 (50%)]	Loss: 0.019893
Train Epoch: 2 [30720/60000 (51%)]	Loss: 0.020163
Train Epoch: 2 [31360/60000 (52%)]	Loss: 0.011787


In [17]:
Perfomance_function(pruned_model)

Model after speedup
Net(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout(p=0.25, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=4608, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

Test set: Average loss: 0.0314, Accuracy: 9910/10000 (99.10%)

+-------+-------+--------+----------------+-----------------+-----------------+---------+---------+
| Index | Name  |  Type  |  Weight Shape  |    Input Size   |   Output Size   |  FLOPs  | #Params |
+-------+-------+--------+----------------+-----------------+-----------------+---------+---------+
|   0   | conv1 | Conv2d | (16, 1, 3, 3)  |  (3, 1, 28, 28) | (3, 16, 26, 26) |  97344  |   160   |
|   1   | conv2 | Conv2d | (32, 16, 3, 3) | (3, 16, 26, 26) | (3, 32, 24, 24) | 2654208 |   4640  |
|   2   | fc1   | Linear |  (128, 4608)   |    (3, 4608)    |     (3, 128) 